In [1]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl
%pip install -U datasets

In [ ]:
#Importing the required Libraries

#Analysis and data creation
import numpy as np
import pandas as pd
import re

#Modelling
import torch
import torch.nn.functional as F

from transformers import (
    MistralForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from tqdm import tqdm

#Quantization
from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel, PeftConfig

#KPM Evaluation
from sklearn.metrics import precision_recall_curve, average_precision_score, precision_score
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import json
import os

In [ ]:
!huggingface-cli login --token $secret_hf

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
pred_dev_df=pd.read_csv("/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/train_dev_test_dataset/final_data/pred_dev_human_gr.csv")
test_df=pd.read_csv("/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/train_dev_test_dataset/final_data/pred_test_gr.csv")

#A function that reads, cleans, preprocess the train, dev and test data

In [ ]:
test_df["kp_arg"] = 'Keypoint: ' + test_df["keypoint"].str.strip() + "; " + 'Επιχείρημα: ' + test_df["argument"].str.strip()
pred_dev_df["kp_arg"] = 'Keypoint: ' + pred_dev_df["keypoint"].str.strip() + "; " + 'Επιχείρημα: ' + pred_dev_df["argument"].str.strip()


test_merged = test_df[["kp_arg"]]
pred_dev_merged=pred_dev_df[["kp_arg"]]

In [ ]:
# Define a function to calculate the length of each entry in 'arg_kp'
def calculate_length(df, column_name):
    return df[column_name].apply(len).max()


test_max_length = calculate_length(test_merged, "kp_arg")
pred_dev_max_length = calculate_length(pred_dev_merged, "kp_arg")


print("Maximum length in test dataset:", test_max_length)
print("Maximum length in pred_dev dataset:", pred_dev_max_length)


Maximum length in train dataset: 438
Maximum length in dev dataset: 399
Maximum length in test dataset: 440
Maximum length in pred_dev dataset: 399


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
peft_model_id = #path to checkpoint/ or/ adapter model of hf"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path, return_dict=True, quantization_config=bnb_config, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.97M [00:00<?, ?B/s]

In [ ]:
# Get the device used by the model
device = next(model.parameters()).device

# Initialize a list to store probabilities
results = []

for text in pred_dev_df["kp_arg"]:
    # Tokenize the input text and move to the same device as the model
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Perform inference
    with torch.no_grad():
        logits = model(**inputs).logits

    # Compute probabilities using softmax
    probabilities = F.softmax(logits, dim=1).squeeze()  # Remove extra dimension

    # Append probabilities as a tuple (class_0, class_1)
    results.append((probabilities[0].item(), probabilities[1].item()))

# Convert the results into a DataFrame
probabilities_df = pd.DataFrame(results, columns=["0", "1"])

# Save the DataFrame to a CSV file
output_file_path = "preds_dev_step645.csv"
probabilities_df.to_csv(output_file_path, index_label="id")

print(f"Probabilities saved to {output_file_path}")

#KPM Evaluation code

In [ ]:
def load_kpm_data(gold_data_dir, subset):

    arguments_file = os.path.join(gold_data_dir, f"arguments_human_translated_{subset}.csv")
    key_points_file = os.path.join(gold_data_dir, f"key_points_human_translated_{subset}.csv")
    labels_file = os.path.join(gold_data_dir, f"labels_{subset}.csv")

    arguments_df = pd.read_csv(arguments_file)
    key_points_df = pd.read_csv(key_points_file)
    labels_file_df = pd.read_csv(labels_file)

    return arguments_df, key_points_df, labels_file_df


def load_predictions(predictions_dir, correct_kp_list):
    arg =[]
    kp = []
    scores = []
    invalid_keypoints = set()
    with open(predictions_dir, "r") as f_in:
        res = json.load(f_in)
        for arg_id, kps in res.items():
            valid_kps = {key: value for key, value in kps.items() if key in correct_kp_list}
            invalid = {key: value for key, value in kps.items() if key not in correct_kp_list}
            for invalid_kp, _ in invalid.items():
                if invalid_kp not in invalid_keypoints:
                    #print(f"key point {invalid_kp} doesn't appear in the key points file and will be ignored")
                    invalid_keypoints.add(invalid_kp)
            if valid_kps:
                best_kp = max(valid_kps.items(), key=lambda x: x[1])
                arg.append(arg_id)
                kp.append(best_kp[0])
                scores.append(best_kp[1])
        #print(f"\tloaded predictions for {len(arg)} arguments")

        return pd.DataFrame({"arg_id" : arg, "key_point_id": kp, "score": scores})

def get_predictions(predictions_file, labels_df, arg_df, kp_df):
    print("\nֿ** loading predictions:")
    arg_df = arg_df[["arg_id", "topic", "stance"]]
    predictions_df = load_predictions(predictions_file, kp_df["key_point_id"].unique())

    #make sure each arg_id has a prediction
    predictions_df = pd.merge(arg_df, predictions_df, how="left", on="arg_id")

    #handle arguements with no matching key point
    predictions_df["key_point_id"] = predictions_df["key_point_id"].fillna("dummy_id")
    predictions_df["score"] = predictions_df["score"].fillna(0)

    #merge each argument with the gold labels
    merged_df = pd.merge(predictions_df, labels_df, how="left", on=["arg_id", "key_point_id"])

    merged_df.loc[merged_df['key_point_id'] == "dummy_id", 'label'] = 0
    merged_df["label_strict"] = merged_df["label"].fillna(0)
    merged_df["label_relaxed"] = merged_df["label"].fillna(1)


    print("\n** predictions analysis:")
    for desc, group in merged_df.groupby(["stance", "topic"]):
        not_dummies = group[group["key_point_id"] != "dummy_id"]
        print(f"\t{desc}:")
        print(f"\t\tsubmitted matched for {len(not_dummies)/len(group):.2} of the arguments ({len(not_dummies)}/{len(group)})")


    return merged_df

def get_ap(df, label_column, top_percentile=0.5):
    top = int(len(df)*top_percentile)
    df = df.sort_values('score', ascending=False).head(top)
    # after selecting top percentile candidates, we set the score for the dummy kp to 1, to prevent it from increasing the precision.
    df.loc[df['key_point_id'] == "dummy_id", 'score'] = 0.99
    return average_precision_score(y_true=df[label_column], y_score=df["score"])

def calc_mean_average_precision(df, label_column):
    precisions = [get_ap(group, label_column) for _, group in df.groupby(["topic", "stance"])]
    return np.mean(precisions)

def evaluate_predictions(merged_df,name = 'train'):
    #print("\n** running evalution:")
    mAP_strict = calc_mean_average_precision(merged_df, "label_strict")
    mAP_relaxed = calc_mean_average_precision(merged_df, "label_relaxed")


    print(f"mAP strict= {mAP_strict} ; mAP relaxed = {mAP_relaxed}")

In [ ]:
predictions = pd.read_csv('/content/preds_dev_step645.csv')
predictions

,id,0,1
0,0,0.986896,0.013104
1,1,0.005137,0.994863
2,2,0.992969,0.007031
3,3,0.993994,0.006006
4,4,0.325191,0.674809
...,...,...,...
4206,4206,0.996240,0.003760
4207,4207,0.786861,0.213139
4208,4208,0.876388,0.123611
4209,4209,0.998916,0.001084


In [ ]:
predictions=predictions['1']
predictions

,1
0,0.013104
1,0.994863
2,0.007031
3,0.006006
4,0.674809
...,...
4206,0.003760
4207,0.213139
4208,0.123611
4209,0.001084


In [ ]:
pred_dev_file_path="/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/train_dev_test_dataset/final_data/pred_dev_human_gr.csv"
test_df = pd.read_csv(pred_dev_file_path)
test_df_selected = test_df[['arg_id', 'key_point_id']]
new_test= pd.concat([test_df_selected, predictions], axis=1)
new_test.rename(columns={'1': 'score'}, inplace=True)
new_test

,arg_id,key_point_id,score
0,arg_4_0,kp_4_0,0.013104
1,arg_4_0,kp_4_1,0.994863
2,arg_4_0,kp_4_2,0.007031
3,arg_4_0,kp_4_3,0.006006
4,arg_4_0,kp_4_4,0.674809
...,...,...,...
4206,arg_15_226,kp_15_3,0.003760
4207,arg_15_226,kp_15_4,0.213139
4208,arg_15_226,kp_15_5,0.123611
4209,arg_15_226,kp_15_6,0.001084


In [ ]:
path_dataset = '/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/train_dev_test_dataset'
path_predictions_folder = '/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/Track_1_KPM/SEQ_CLS/Finetuning_Meltemi_base_for SEQ_CLS/3. meltemi_base_peft_without_topic_weighted_classes/preds/json_preds/'

In [ ]:
arg_df, kp_df, labels_df = load_kpm_data(path_dataset, subset="dev")

In [ ]:
save_predictions_name = 'seq_cls_meltemi_7b_base'  ##this one needs to be more dynamic
args = {}
kps = {}

for arg,kp,score in zip(new_test['arg_id'],new_test['key_point_id'],new_test['score']):
  args[arg] = {}
for arg,kp,score in zip(new_test['arg_id'],new_test['key_point_id'],new_test['score']):
  args[arg][kp] = score

with open(path_predictions_folder + save_predictions_name + '_' + 'predictions_without_topic_human_dev.p.', 'w') as fp:
  fp.write(json.dumps(args))
  fp.close()

In [ ]:
merged_df = get_predictions(path_predictions_folder + save_predictions_name + '_' + 'predictions_without_topic_human_dev.p.', labels_df, arg_df, kp_df)
merged_df.to_csv('/content/drive/MyDrive/Πτυχιακή/Code/Experiments_Meltemi/Track_1_KPM/SEQ_CLS/Finetuning_Meltemi_base_for SEQ_CLS/3. meltemi_base_peft_without_topic_weighted_classes/preds/merged_df/merged_df_cls_without_topic_human_dev.csv', index=False)


ֿ** loading predictions:

** predictions analysis:
	(-1, 'Θα πρέπει να καταργηθεί η μαθητική στολή στα σχολεία'):
		submitted matched for 1.0 of the arguments (121/121)
	(-1, 'Θα πρέπει να καταργηθεί το δικαίωμα οπλοκατοχής και οπλοφορίας'):
		submitted matched for 1.0 of the arguments (123/123)
	(-1, 'Θα πρέπει να σταματήσει η θετική δράση'):
		submitted matched for 1.0 of the arguments (108/108)
	(-1, 'Θα πρέπει να υιοθετήσουμε καθεστώς λιτότητας'):
		submitted matched for 1.0 of the arguments (108/108)
	(1, 'Θα πρέπει να καταργηθεί η μαθητική στολή στα σχολεία'):
		submitted matched for 1.0 of the arguments (117/117)
	(1, 'Θα πρέπει να καταργηθεί το δικαίωμα οπλοκατοχής και οπλοφορίας'):
		submitted matched for 1.0 of the arguments (110/110)
	(1, 'Θα πρέπει να σταματήσει η θετική δράση'):
		submitted matched for 1.0 of the arguments (119/119)
	(1, 'Θα πρέπει να υιοθετήσουμε καθεστώς λιτότητας'):
		submitted matched for 1.0 of the arguments (126/126)


In [ ]:
evaluate_predictions(merged_df)

mAP strict= 0.7559238429676498 ; mAP relaxed = 0.9283231464404917


#Merge the adapter weights to the base LLM and push to hf hub

In [ ]:
# Save trained model
#trainer.model.save_pretrained("meltemi_base_finetuning_kpm_kp_arg")

#from peft import AutoPeftModelForSequenceClassification

#fine_tuned_model=AutoPeftModelForSequenceClassification.from_pretrained(
#    '/content/meltemi_base_finetuning_kpm_kp_arg',
#    torch_dtype=torch.bfloat16,
#    trust_remote_code=True)

#merged_model=fine_tuned_model.merge_and_unload()

#HF_USERNAME="Kleo"
#HF_REPO_NAME="Meltemi_7b_v1_base_finetuned_seq_cls_kpm_kp_arg_input"
#merged_model.push_to_hub(f"{HF_USERNAME}/{HF_REPO_NAME}")
#tokenizer.push_to_hub(f"{HF_USERNAME}/{HF_REPO_NAME}")